In [77]:
import pandas as pd
import numpy as np
import pylab
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from datetime import datetime, date
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_val_score,StratifiedKFold,KFold
import warnings
import imblearn
from imblearn.combine import SMOTETomek
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
warnings.filterwarnings('ignore')

In [2]:
# today = date.today()

def age(born):
#     born = datetime.strptime(born, "%Y/%m/%d").date()
    today = date.today()
    return today.year - born.year - ((today.month, 
                                      today.day) < (born.month, 
                                                    born.day))


def dataframe_check_suite(x):
    
    
    print("SIZE OF THE DATAFRAME")
    print(x.size)
    print("SHAPE OF THE DATAFRAME")
    print(x.shape)
    
    print("BELOW ARE THE UNIQUE VALUES IN EVERY COLUMN")
    for col in x:
        print(col,end = " : ")
        print(x[col].unique())
        print("")
    print("******************************************************************************")
    print("BELOW  SUM OF THE NULL VALUES IN EVERY COLUMN")
    col_null=[]
    for i in x.columns:
#     print(i)
        if x[i].isnull().sum()>0:
            print(i,"",x[i].isnull().sum(), end="  : ")
#         print(":")
            col_null.append(i)
    print(col_null)
    print(len(col_null))

    

In [3]:
def yobla(X_train, X_test, y_train, y_test):
    
    log_reg_params = [{"C":0.001}, {"C":0.1}, {"C":1}, {"C":10}]
    dec_tree_params = [{"criterion": "gini"}, {"criterion": "entropy"}]
    rand_for_params = [{"criterion": "gini"}, {"criterion": "entropy"}]
    kneighbors_params = [{"n_neighbors":3}, {"n_neighbors":5}]
    naive_bayes_params = [{}]
    svc_params = [{"C":0.1}, {"C":1}, {"C":10}, {"C":100}]

    modelclasses = [
        ["log regression", LogisticRegression, log_reg_params],
        ["decision tree", DecisionTreeClassifier, dec_tree_params],
        ["random forest", RandomForestClassifier, rand_for_params],
        ["k neighbors", KNeighborsClassifier, kneighbors_params],
        ["naive bayes", GaussianNB, naive_bayes_params],
        ["support vector machines", SVC, svc_params]
    ]


    insights = []
    for modelname, Model, params_list in modelclasses:
        for params in params_list:
            model = Model(**params)
            model.fit(X_train, y_train)
            y_pred =model.predict(X_test)
            score = model.score(X_test, y_test)

            f_score =f1_score(y_test, y_pred, average="macro")
            pre_score= precision_score(y_test, y_pred, average="macro")
            rec_score =recall_score(y_test, y_pred, average="macro")
            insights.append((modelname, model, params, score,"F1 Score:->",f_score,"PRE Score:->",pre_score,"REC Score:->",rec_score))



    for i in insights:
        print(i)

In [4]:
# df = pd.read_excel("Mental Health Survey under Centre for Development for Advanced Computing, Noida (Responses).xlsx")
df = pd.read_csv("Mental Health Survey under Centre for Development for Advanced Computing, Noida.csv")




In [5]:

df = df.iloc[:,1:]
List_change = ["Energy","Substance Craving ",
               "Sadness","Anger","Exercising",
               "Irritability" ]
List_change = [x.upper() for x in List_change]
list_to_change =list(df.columns)
list_to_change = [x.upper() for x in list_to_change]



j=0
r = 1
for i in range(len(list_to_change)):
    
        r=1
        if list_to_change[i].startswith("DAY 1"):
            while r<=7:
                
                k = " DAY "+ str(r)
                r=int(r)+1
                list_to_change[i]=List_change[j]+k
                i=i+1

            j=j+1
# print(list_to_change)       
df.columns = list_to_change  


list_cj=[]
for i in df.columns:
#     print(i)
    i =i.replace(",", "COMMA")
#     print(i)
    url = "_".join( i.split() )
#     print(url)
    list_cj.append(url)
df.columns = list_cj
df.replace(to_replace ="Most of the Time/Always",
                 value ="Most of the Time",inplace = True)
df["DATE_OF_BIRTH"]=pd.to_datetime(df["DATE_OF_BIRTH"], format='%Y/%m/%d')
# df.rename(columns = {'AGE' : 'Date of Birth'}, inplace = True)
df_q = df.iloc[:,0:27]
del df_q["OKAY?"]
df_s = df.iloc[:,27:]

In [6]:
dataframe_check_suite(df_q)

SIZE OF THE DATAFRAME
2418
SHAPE OF THE DATAFRAME
(93, 26)
BELOW ARE THE UNIQUE VALUES IN EVERY COLUMN
NAME : ['Khushboo Kumari' 'ND' 'Priyanshi Tyagi ' 'Shashwat shah'
 'Vallabhi Shah ' 'Aditi' 'LAvanya Bharti' 'Vaibhav Gupta' 'Pooja Sharma '
 'Rupal Jain ' 'Anshika Arora' 'Akash Jatwa' 'Rohit Machiwal ' 'Chandni '
 'Aiysha' 'Dheeraj Chandra' 'Vaibhav' 'Udit' 'Divya' 'Jyoti '
 'Shashwat Singh' 'Aakanksha' 'Himanshu singh ' 'Diksha ' 'Varnika'
 'Priya ' 'Vinayak' 'Aditya Kumar ' 'Ephraim Elow Chalai' 'Vrinda '
 'Kaushal keshri ' 'Rahul Rathour' 'Abhishek Biswas' 'Surbhi Gaur'
 'Kamna Mamgain ' 'Nitish' 'Nalin Gupta' 'Anupam ' 'Ajay kumar'
 'Shaifali Bhatt' 'Bhavya kukreja ' 'Lokesh Jain' 'Shubham Chaudhary'
 'Mohit Arora' 'Deeksha Aashri' 'Kushagra bhardwaj ' 'Kanika Aggarwal '
 'Akhil Tripathi' 'Shrikant' 'Geetika' 'Prashant Kumar ' 'Nikhil'
 'Akhil pratap singh' 'Vidushi' 'Prateek' 'Shubham pal' 'Vikas Pal'
 'Abhishek Singh' 'Riya chauhan' 'Indresh shukla' 'Sumit Bhati ' 'Sanju'
 'Ma

In [7]:
df_q.fillna('Never', inplace=True)

accepted_strings = ["Most of the Time", "Quite Often", "Less than Often", "Never"]
h_ls=df_q.columns[df_q.isin(accepted_strings).any()].tolist()
n_h_ls=[]
for i in df_q.columns:
    if i not in h_ls:
        n_h_ls.append(i)
len(n_h_ls)
size_mapping = {"Most of the Time":7, "Quite Often":3, "Less than Often":-1, "Never":-5}
for i in df_q.columns:
    if i in h_ls:
        df_q[i]=df_q[i].map(size_mapping)


In [8]:

size_mapping_s = {"Most of the Time":int(7), "Quite Often":int(3), "Sometimes":int(-1), "Never":int(-3),'No workout':int(5) ,
                  '10-15':int (-11), '60+ mins':int (-6) ,'30-60 mins':int (-6) ,'10-15 mins':int (-1)}
for i in df_s.columns:
#     if i in h_ls:
    df_s[i]=df_s[i].map(size_mapping_s)

# Imputations using median of the values in row  
# df_check= df_s.apply(lambda row: row.fillna(row.median()), axis=1)
df_s = df_s.T.fillna(df_s.median(axis=1)).T
df_s=df_s.astype(int)
df_s[['NAME', 'DATE_OF_BIRTH',
      'GENDER',
      'FEELING_NERVOUSCOMMA_ANXIOUSCOMMA_OR_ON_EDGE']] = df_q[['NAME', 'DATE_OF_BIRTH', 
                                                                 'GENDER',
                                                            'FEELING_NERVOUSCOMMA_ANXIOUSCOMMA_OR_ON_EDGE']].to_numpy()

In [9]:
df_cc = pd.merge(df_q, df_s, on=['NAME', 'DATE_OF_BIRTH','GENDER','FEELING_NERVOUSCOMMA_ANXIOUSCOMMA_OR_ON_EDGE'])
# df_c.shape
df_cc['GENDER']=df_cc['GENDER'].str.upper()
one_hot=pd.get_dummies(df_cc["GENDER"])

data_cc = df_cc.drop('GENDER',axis = 1)

df_cc = data_cc.join(one_hot)

# data_df 

In [10]:
df_cc_gad = df_cc[['NAME', 'DATE_OF_BIRTH', 'FEMALE', 'MALE',"PREFER NOT TO SAY",
       'HAVE_YOU_BEEN_DIAGNOSED_FOR_ANXIETY/ANXIETY_RELATED_DISORDERS_(_BY_A_PROFESSIONAL_:_THERAPISTCOMMA_DOCTOR_ETC._)',
                  'FEELING_NERVOUSCOMMA_ANXIOUSCOMMA_OR_ON_EDGE',
       'NOT_BEING_ABLE_TO_STOP_OR_CONTROL_WORRYING',
       'WORRYING_TOO_MUCH_ABOUT_DIFFERENT_THINGS', 'TROUBLE_RELAXING',
       'BEING_SO_RESTLESS_THAT_IT_IS_HARD_TO_SIT_STILL',
       'BECOMING_EASILY_ANNOYED_OR_IRRITABLE',
       'FEELING_AFRAIDCOMMA_AS_IF_SOMETHING_AWFUL_MIGHT_HAPPEN']]



In [11]:
df_q['GENDER']=df_q['GENDER'].str.upper()
one_hot=pd.get_dummies(df_q["GENDER"])

data_q = df_q.drop('GENDER',axis = 1)

df_q = data_q.join(one_hot)

In [12]:
df_cc["ANXIETY"] = df_cc["FEELING_NERVOUSCOMMA_ANXIOUSCOMMA_OR_ON_EDGE"]
del df_cc["FEELING_NERVOUSCOMMA_ANXIOUSCOMMA_OR_ON_EDGE"]

df_s["ANXIETY"] = df_s["FEELING_NERVOUSCOMMA_ANXIOUSCOMMA_OR_ON_EDGE"]
del df_s["FEELING_NERVOUSCOMMA_ANXIOUSCOMMA_OR_ON_EDGE"]
df_q["ANXIETY"] = df_q["FEELING_NERVOUSCOMMA_ANXIOUSCOMMA_OR_ON_EDGE"]
del df_q["FEELING_NERVOUSCOMMA_ANXIOUSCOMMA_OR_ON_EDGE"]

df_cc_gad["ANXIETY"] = df_cc_gad["FEELING_NERVOUSCOMMA_ANXIOUSCOMMA_OR_ON_EDGE"]
del df_cc_gad["FEELING_NERVOUSCOMMA_ANXIOUSCOMMA_OR_ON_EDGE"]

df_cc['AGE'] = df_cc['DATE_OF_BIRTH'].apply(age)
del df_cc["DATE_OF_BIRTH"]
df_q['AGE'] = df_q['DATE_OF_BIRTH'].apply(age)
del df_q["DATE_OF_BIRTH"]
df_s['AGE'] = df_s['DATE_OF_BIRTH'].apply(age)
del df_s["DATE_OF_BIRTH"]



df_cc_gad['AGE'] = df_cc_gad['DATE_OF_BIRTH'].apply(age)
del df_cc_gad["DATE_OF_BIRTH"]



# del df_cc['HAVE_YOU_BEEN_DIAGNOSED_FOR_ANXIETY/ANXIETY_RELATED_DISORDERS_(_BY_A_PROFESSIONAL_:_THERAPISTCOMMA_DOCTOR_ETC._)']
# del df_q['HAVE_YOU_BEEN_DIAGNOSED_FOR_ANXIETY/ANXIETY_RELATED_DISORDERS_(_BY_A_PROFESSIONAL_:_THERAPISTCOMMA_DOCTOR_ETC._)']
# dataframe_check_suite(df_cc)

In [13]:
size_mapping_s = {"Yes":int(10), "No":int(-2), "Prefer not to say":int(2)}


df_cc['HAVE_YOU_BEEN_DIAGNOSED_FOR_ANXIETY/ANXIETY_RELATED_DISORDERS_(_BY_A_PROFESSIONAL_:_THERAPISTCOMMA_DOCTOR_ETC._)']=df_cc['HAVE_YOU_BEEN_DIAGNOSED_FOR_ANXIETY/ANXIETY_RELATED_DISORDERS_(_BY_A_PROFESSIONAL_:_THERAPISTCOMMA_DOCTOR_ETC._)'].map(size_mapping_s)



df_q['HAVE_YOU_BEEN_DIAGNOSED_FOR_ANXIETY/ANXIETY_RELATED_DISORDERS_(_BY_A_PROFESSIONAL_:_THERAPISTCOMMA_DOCTOR_ETC._)']=df_q['HAVE_YOU_BEEN_DIAGNOSED_FOR_ANXIETY/ANXIETY_RELATED_DISORDERS_(_BY_A_PROFESSIONAL_:_THERAPISTCOMMA_DOCTOR_ETC._)'].map(size_mapping_s)

df_cc_gad['HAVE_YOU_BEEN_DIAGNOSED_FOR_ANXIETY/ANXIETY_RELATED_DISORDERS_(_BY_A_PROFESSIONAL_:_THERAPISTCOMMA_DOCTOR_ETC._)']=df_cc_gad['HAVE_YOU_BEEN_DIAGNOSED_FOR_ANXIETY/ANXIETY_RELATED_DISORDERS_(_BY_A_PROFESSIONAL_:_THERAPISTCOMMA_DOCTOR_ETC._)'].map(size_mapping_s)




In [14]:
df_s['GENDER']=df_s['GENDER'].str.upper()
one_hot=pd.get_dummies(df_s["GENDER"])

data_q = df_s.drop('GENDER',axis = 1)

df_s = data_q.join(one_hot)

In [15]:
X = df_cc.drop(['ANXIETY','NAME'],axis =1)  #independent columns
y = df_cc[['ANXIETY']]    #target column i.e price range

X_q = df_q.drop(['ANXIETY','NAME'],axis =1)  #independent columns
y_q = df_q[['ANXIETY']]    #target column i.e price range

X_s = df_s.drop(['ANXIETY','NAME'],axis =1)  #independent columns
y_s = df_s[['ANXIETY']]    #target column i.e price range

X_s = df_s.drop(['ANXIETY','NAME'],axis =1)  #independent columns
y_s = df_s[['ANXIETY']]    #target column i.e price range

In [16]:
X_cc_gad = df_cc_gad.drop(['ANXIETY','NAME'],axis =1)  #independent columns
y_cc_gad = df_cc_gad[['ANXIETY']]    #target column i.e price range

In [18]:
yobla(X_train, X_test, y_train, y_test)

('log regression', LogisticRegression(C=0.001), {'C': 0.001}, 0.6842105263157895, 'F1 Score:->', 0.3827375762859634, 'PRE Score:->', 0.3431372549019608, 'REC Score:->', 0.4330357142857143)
('log regression', LogisticRegression(C=0.1), {'C': 0.1}, 0.5789473684210527, 'F1 Score:->', 0.5077188940092165, 'PRE Score:->', 0.515625, 'REC Score:->', 0.5044642857142857)
('log regression', LogisticRegression(C=1), {'C': 1}, 0.5526315789473685, 'F1 Score:->', 0.4817610396030863, 'PRE Score:->', 0.48333333333333334, 'REC Score:->', 0.48660714285714285)
('log regression', LogisticRegression(C=10), {'C': 10}, 0.6052631578947368, 'F1 Score:->', 0.5432676518883415, 'PRE Score:->', 0.536904761904762, 'REC Score:->', 0.5669642857142857)
('decision tree', DecisionTreeClassifier(), {'criterion': 'gini'}, 0.7105263157894737, 'F1 Score:->', 0.6932471264367815, 'PRE Score:->', 0.7589743589743589, 'REC Score:->', 0.6830357142857143)
('decision tree', DecisionTreeClassifier(criterion='entropy'), {'criterion': 

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
yobla(X_train, X_test, y_train, y_test)


('log regression', LogisticRegression(C=0.001), {'C': 0.001}, 0.75, 'F1 Score:->', 0.4222222222222223, 'PRE Score:->', 0.37857142857142856, 'REC Score:->', 0.4772727272727273)
('log regression', LogisticRegression(C=0.1), {'C': 0.1}, 0.5, 'F1 Score:->', 0.36507936507936506, 'PRE Score:->', 0.3772893772893773, 'REC Score:->', 0.38446969696969696)
('log regression', LogisticRegression(C=1), {'C': 1}, 0.5416666666666666, 'F1 Score:->', 0.49722222222222223, 'PRE Score:->', 0.511904761904762, 'REC Score:->', 0.509469696969697)
('log regression', LogisticRegression(C=10), {'C': 10}, 0.5, 'F1 Score:->', 0.4640756302521009, 'PRE Score:->', 0.47916666666666663, 'REC Score:->', 0.48674242424242425)
('decision tree', DecisionTreeClassifier(), {'criterion': 'gini'}, 0.25, 'F1 Score:->', 0.14285714285714285, 'PRE Score:->', 0.11538461538461539, 'REC Score:->', 0.1875)
('decision tree', DecisionTreeClassifier(criterion='entropy'), {'criterion': 'entropy'}, 0.2916666666666667, 'F1 Score:->', 0.183333

In [25]:
X.shape

(93, 68)

In [26]:
y.shape

(93, 1)

In [31]:
smt = SMOTETomek(random_state=42)
X_smote, y_smote =smt.fit_resample(X, y)

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.4, random_state=42)
yobla(X_train, X_test, y_train, y_test)

('log regression', LogisticRegression(C=0.001), {'C': 0.001}, 0.6666666666666666, 'F1 Score:->', 0.6518775987525988, 'PRE Score:->', 0.6603203083466241, 'REC Score:->', 0.6588522588522588)
('log regression', LogisticRegression(C=0.1), {'C': 0.1}, 0.8, 'F1 Score:->', 0.773117749123941, 'PRE Score:->', 0.8315934065934066, 'REC Score:->', 0.782051282051282)
('log regression', LogisticRegression(C=1), {'C': 1}, 0.8, 'F1 Score:->', 0.773117749123941, 'PRE Score:->', 0.8315934065934066, 'REC Score:->', 0.782051282051282)
('log regression', LogisticRegression(C=10), {'C': 10}, 0.75, 'F1 Score:->', 0.7182354768561665, 'PRE Score:->', 0.7934894053315106, 'REC Score:->', 0.7322649572649573)
('decision tree', DecisionTreeClassifier(), {'criterion': 'gini'}, 0.6333333333333333, 'F1 Score:->', 0.6323887395459976, 'PRE Score:->', 0.6460813492063492, 'REC Score:->', 0.6363858363858363)
('decision tree', DecisionTreeClassifier(criterion='entropy'), {'criterion': 'entropy'}, 0.6666666666666666, 'F1 Sco

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.3, random_state=42)
yobla(X_train, X_test, y_train, y_test)

('log regression', LogisticRegression(C=0.001), {'C': 0.001}, 0.7111111111111111, 'F1 Score:->', 0.706159420289855, 'PRE Score:->', 0.72991452991453, 'REC Score:->', 0.7382617382617382)
('log regression', LogisticRegression(C=0.1), {'C': 0.1}, 0.8666666666666667, 'F1 Score:->', 0.8606125356125356, 'PRE Score:->', 0.8927489177489178, 'REC Score:->', 0.8571428571428572)
('log regression', LogisticRegression(C=1), {'C': 1}, 0.8222222222222222, 'F1 Score:->', 0.8113701113701113, 'PRE Score:->', 0.8217261904761906, 'REC Score:->', 0.8392857142857143)
('log regression', LogisticRegression(C=10), {'C': 10}, 0.8, 'F1 Score:->', 0.7865384615384614, 'PRE Score:->', 0.8017857142857142, 'REC Score:->', 0.8214285714285714)
('decision tree', DecisionTreeClassifier(), {'criterion': 'gini'}, 0.5777777777777777, 'F1 Score:->', 0.5789455110107284, 'PRE Score:->', 0.6339190221543163, 'REC Score:->', 0.6035214785214785)
('decision tree', DecisionTreeClassifier(criterion='entropy'), {'criterion': 'entropy'

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.25, random_state=42)
yobla(X_train, X_test, y_train, y_test)

('log regression', LogisticRegression(C=0.001), {'C': 0.001}, 0.7368421052631579, 'F1 Score:->', 0.7332915622389305, 'PRE Score:->', 0.7625811688311689, 'REC Score:->', 0.7506493506493507)
('log regression', LogisticRegression(C=0.1), {'C': 0.1}, 0.868421052631579, 'F1 Score:->', 0.8482213438735178, 'PRE Score:->', 0.8875, 'REC Score:->', 0.8428571428571429)
('log regression', LogisticRegression(C=1), {'C': 1}, 0.868421052631579, 'F1 Score:->', 0.853790945896209, 'PRE Score:->', 0.8611111111111112, 'REC Score:->', 0.8535714285714285)
('log regression', LogisticRegression(C=10), {'C': 10}, 0.8421052631578947, 'F1 Score:->', 0.8288690476190476, 'PRE Score:->', 0.8511904761904762, 'REC Score:->', 0.8392857142857143)
('decision tree', DecisionTreeClassifier(), {'criterion': 'gini'}, 0.631578947368421, 'F1 Score:->', 0.6229102167182663, 'PRE Score:->', 0.6333333333333333, 'REC Score:->', 0.6246753246753247)
('decision tree', DecisionTreeClassifier(criterion='entropy'), {'criterion': 'entrop

In [44]:
df_cc['ANXIETY'].unique()

array([ 3, -1, -5,  7], dtype=int64)

In [36]:

print(Counter(y))

Counter({'ANXIETY': 1})


In [57]:

log_reg_params = [{"C":0.001}, {"C":0.01}, {"C":1}, {"C":10}]
dec_tree_params = [{"criterion": "gini"}, {"criterion": "entropy"}]
rand_for_params = [{"criterion": "gini"}, {"criterion": "entropy"}]
kneighbors_params = [{"n_neighbors":3}, {"n_neighbors":5}]
svc_params = [{"C":0.1}, {"C":1}, {"C":10}, {"C":100}]

modelclasses = [
    ["log regression", LogisticRegression, log_reg_params],   
    ["random forest", RandomForestClassifier, rand_for_params],
    ["decision tree", DecisionTreeClassifier, dec_tree_params],
    ["support vector machines", SVC, svc_params]
]
insights = []
for modelname, Model, params_list in modelclasses:
    for params in params_list:
        kf=KFold(n_splits=10)

        model = Model(**params)

        score=cross_val_score(model,X,y,cv=kf)
        print("MODEL NAME is ",modelname," MODEL PARAMETER IS ", params)
        print("Cross Validation Scores are {}".format(score))
        print("Average Cross Validation score :{}".format(score.mean()))




MODEL NAME is  log regression  MODEL PARAMETER IS  {'C': 0.001}
Cross Validation Scores are [0.5        0.6        0.7        0.77777778 0.44444444 0.44444444
 0.55555556 0.77777778 0.44444444 0.77777778]
Average Cross Validation score :0.6022222222222223
MODEL NAME is  log regression  MODEL PARAMETER IS  {'C': 0.01}
Cross Validation Scores are [0.6        0.6        0.7        0.55555556 0.55555556 0.55555556
 0.55555556 0.77777778 0.55555556 0.55555556]
Average Cross Validation score :0.6011111111111112
MODEL NAME is  log regression  MODEL PARAMETER IS  {'C': 1}
Cross Validation Scores are [0.6        0.6        0.6        0.55555556 0.66666667 0.44444444
 0.55555556 0.44444444 0.55555556 0.44444444]
Average Cross Validation score :0.5466666666666666
MODEL NAME is  log regression  MODEL PARAMETER IS  {'C': 10}
Cross Validation Scores are [0.6        0.5        0.6        0.44444444 0.55555556 0.33333333
 0.55555556 0.55555556 0.55555556 0.44444444]
Average Cross Validation score :0.5

In [58]:
log_reg_params = [{"C":0.001}, {"C":0.01}, {"C":1}, {"C":10}]
dec_tree_params = [{"criterion": "gini"}, {"criterion": "entropy"}]
rand_for_params = [{"criterion": "gini"}, {"criterion": "entropy"}]
kneighbors_params = [{"n_neighbors":3}, {"n_neighbors":5}]
svc_params = [{"C":0.1}, {"C":1}, {"C":10}, {"C":100}]

modelclasses = [
    ["log regression", LogisticRegression, log_reg_params],   
    ["random forest", RandomForestClassifier, rand_for_params],
    ["decision tree", DecisionTreeClassifier, dec_tree_params],
    ["support vector machines", SVC, svc_params]
]
insights = []
for modelname, Model, params_list in modelclasses:
    for params in params_list:
        kf=KFold(n_splits=10)

        model = Model(**params)

        score=cross_val_score(model,X_smote, y_smote,cv=kf)
        print("MODEL NAME is ",modelname," MODEL PARAMETER IS ", params)
        print("Cross Validation Scores are {}".format(score))
        print("Average Cross Validation score :{}".format(score.mean()))




MODEL NAME is  log regression  MODEL PARAMETER IS  {'C': 0.001}
Cross Validation Scores are [0.53333333 0.73333333 0.53333333 0.46666667 0.6        0.6
 0.86666667 0.8        0.6        0.73333333]
Average Cross Validation score :0.6466666666666666
MODEL NAME is  log regression  MODEL PARAMETER IS  {'C': 0.01}
Cross Validation Scores are [0.6        0.8        0.66666667 0.66666667 0.53333333 0.66666667
 0.93333333 1.         1.         1.        ]
Average Cross Validation score :0.7866666666666667
MODEL NAME is  log regression  MODEL PARAMETER IS  {'C': 1}
Cross Validation Scores are [0.73333333 0.73333333 0.6        0.8        0.66666667 0.53333333
 0.93333333 1.         1.         1.        ]
Average Cross Validation score :0.8
MODEL NAME is  log regression  MODEL PARAMETER IS  {'C': 10}
Cross Validation Scores are [0.73333333 0.73333333 0.6        0.73333333 0.66666667 0.53333333
 0.93333333 1.         1.         1.        ]
Average Cross Validation score :0.7933333333333333
MODEL 

In [76]:
y['ANXIETY'].value_counts()

 3    38
-1    35
 7    10
-5    10
Name: ANXIETY, dtype: int64

In [74]:
y_smote['ANXIETY'].value_counts()

 7    38
 3    38
-5    37
-1    37
Name: ANXIETY, dtype: int64

# UNDERSAMPLING


In [78]:
# instantiating the random undersampler
rus = RandomUnderSampler() 
# resampling X, y
X_rus, y_rus = rus.fit_resample(X, y)
# new class distribution
print(Counter(y_rus))

Counter({'ANXIETY': 1})


In [79]:
y_rus['ANXIETY'].value_counts()

 7    10
-5    10
 3    10
-1    10
Name: ANXIETY, dtype: int64

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X_rus, y_rus, test_size=0.4, random_state=42)
yobla(X_train, X_test, y_train, y_test)

('log regression', LogisticRegression(C=0.001), {'C': 0.001}, 0.375, 'F1 Score:->', 0.3088023088023088, 'PRE Score:->', 0.26785714285714285, 'REC Score:->', 0.3958333333333333)
('log regression', LogisticRegression(C=0.1), {'C': 0.1}, 0.625, 'F1 Score:->', 0.5928571428571429, 'PRE Score:->', 0.6041666666666666, 'REC Score:->', 0.6083333333333334)
('log regression', LogisticRegression(C=1), {'C': 1}, 0.625, 'F1 Score:->', 0.5928571428571429, 'PRE Score:->', 0.6041666666666666, 'REC Score:->', 0.6083333333333334)
('log regression', LogisticRegression(C=10), {'C': 10}, 0.625, 'F1 Score:->', 0.5928571428571429, 'PRE Score:->', 0.6041666666666666, 'REC Score:->', 0.6083333333333334)
('decision tree', DecisionTreeClassifier(), {'criterion': 'gini'}, 0.5625, 'F1 Score:->', 0.5284090909090909, 'PRE Score:->', 0.5714285714285714, 'REC Score:->', 0.525)
('decision tree', DecisionTreeClassifier(criterion='entropy'), {'criterion': 'entropy'}, 0.375, 'F1 Score:->', 0.32900432900432897, 'PRE Score:-